In [2]:
!wget https://gist.githubusercontent.com/blakesanie/dde3a2b7e698f52f389532b4b52bc254/raw/76fe1b5e9efcf0d2afdfd78b0bfaa737ad0a67d3/shakespeare.txt

--2025-06-27 13:45:28--  https://gist.githubusercontent.com/blakesanie/dde3a2b7e698f52f389532b4b52bc254/raw/76fe1b5e9efcf0d2afdfd78b0bfaa737ad0a67d3/shakespeare.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5436475 (5.2M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   5.18M  --.-KB/s    in 0.06s   

2025-06-27 13:45:29 (85.5 MB/s) - ‘shakespeare.txt’ saved [5436475/5436475]



In [3]:
import re
from collections import Counter
import numpy as np
import nltk

# Завантажуємо корпус Gutenberg, що містить твори Шекспіра
nltk.download('gutenberg')
from nltk.corpus import gutenberg

print("Бібліотеки та дані NLTK готові до роботи.")

[nltk_data] Downloading package gutenberg to /root/nltk_data...


Бібліотеки та дані NLTK готові до роботи.


[nltk_data]   Unzipping corpora/gutenberg.zip.


In [8]:
import re
from collections import Counter

def process_local_file(filepath):
    """
    Читає текстовий файл, очищує його та повертає список слів.
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read()
    except FileNotFoundError:
        print(f"Помилка: файл '{filepath}' не знайдено. Переконайтеся, що ви завантажили його і назва вказана правильно.")
        return []

    # Приводимо текст до нижнього регістру
    text = text.lower()

    # Використовуємо регулярний вираз, щоб знайти всі послідовності літер (слова)
    # Це краще, ніж split(), оскільки правильно обробляє пунктуацію
    words = re.findall(r'\b\w+\b', text)

    return words

# --- ОСНОВНА ЧАСТИНА ---


my_shakespeare_file = 'shakespeare.txt'

# Викликаємо нову функцію для обробки вашого файлу
words = process_local_file(my_shakespeare_file)

if words:
    # Створюємо словник частотності слів
    freqs = Counter(words)
    vocab = set(words)
    total_words = sum(freqs.values())

    print(f"Ваш корпус успішно завантажено. Розмір словника (унікальних слів): {len(vocab)}")
    print(f"Загальна кількість слів у корпусі: {total_words}")
    print("\nПриклад 5 найпоширеніших слів:")
    print(freqs.most_common(5))
else:
    print("Обробку не виконано через помилку читання файлу.")

Ваш корпус успішно завантажено. Розмір словника (унікальних слів): 23723
Загальна кількість слів у корпусі: 927610

Приклад 5 найпоширеніших слів:
[('the', 27594), ('and', 26735), ('i', 22538), ('to', 19771), ('of', 18132)]


In [9]:
def get_probability(word, freqs, total_words):
    """Розраховує ймовірність появи слова в корпусі."""
    return freqs.get(word, 0) / total_words

# Приклад
word_example = 'hamlet'
prob_example = get_probability(word_example, freqs, total_words)
print(f"Ймовірність слова '{word_example}': {prob_example:.6f}")

Ймовірність слова 'hamlet': 0.000126


In [10]:
def get_edits_one(word):
    """
    Генерує всі можливі слова на відстані одного редагування.
    Повертає set() унікальних слів.
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    # Видалення (Delete)
    deletes = [L + R[1:] for L, R in splits if R]

    # Перестановка (Switch/Transpose)
    switches = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]

    # Заміна (Replace)
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

    # Вставка (Insert)
    inserts = [L + c + R for L, R in splits for c in letters]

    return set(deletes + switches + replaces + inserts)

# Приклад
example_word_edit = 'lov'
edits = get_edits_one(example_word_edit)
print(f"Згенеровано {len(edits)} варіантів на відстані 1 від слова '{example_word_edit}'.")
print(f"Приклади: {list(edits)[:5]}")

Згенеровано 182 варіантів на відстані 1 від слова 'lov'.
Приклади: ['lozv', 'loc', 'dov', 'lox', 'hlov']


In [11]:
def get_edits_two(word):
    """Генерує всі можливі слова на відстані двох редагувань."""
    return {e2 for e1 in get_edits_one(word) for e2 in get_edits_one(e1)}

def get_candidates(word, vocab):
    """
    Повертає список можливих слів-кандидатів для виправлення.
    Пріоритет: 1) Оригінальне слово, 2) Слова на відстані 1, 3) Слова на відстані 2.
    """
    # Якщо слово правильне, повертаємо його
    if word in vocab:
        return {word}

    # Шукаємо кандидатів на відстані 1
    candidates_d1 = get_edits_one(word)
    known_candidates_d1 = candidates_d1.intersection(vocab)
    if known_candidates_d1:
        return known_candidates_d1

    # Якщо на відстані 1 нічого немає, шукаємо на відстані 2
    candidates_d2 = get_edits_two(word)
    known_candidates_d2 = candidates_d2.intersection(vocab)
    if known_candidates_d2:
        return known_candidates_d2

    # Якщо нічого не знайдено, повертаємо оригінальне слово
    return {word}

# Приклад
misspelled_word = 'kingdmoe' # Помилка у слові 'kingdome'
candidates = get_candidates(misspelled_word, vocab)
print(f"Знайдені кандидати для '{misspelled_word}': {candidates}")

Знайдені кандидати для 'kingdmoe': {'kingdom', 'kingdoms'}


In [12]:
def min_edit_distance(source, target):
    """Обчислює мінімальну відстань редагування між двома словами."""
    m, n = len(source), len(target)
    dp = np.zeros((m + 1, n + 1), dtype=int)

    for i in range(m + 1):
        dp[i, 0] = i
    for j in range(n + 1):
        dp[0, j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = 0 if source[i - 1] == target[j - 1] else 1
            dp[i, j] = min(dp[i - 1, j] + 1,        # Deletion
                           dp[i, j - 1] + 1,        # Insertion
                           dp[i - 1, j - 1] + cost) # Substitution

    return dp[m, n]

# Приклад
dist = min_edit_distance("apple", "apply")
print(f"Мінімальна відстань редагування між 'apple' та 'apply': {dist}")

Мінімальна відстань редагування між 'apple' та 'apply': 1


In [13]:
def autocorrect(word, vocab, freqs, total_words):
    """
    Основна функція автокорекції.
    Знаходить найкраще виправлення для слова.
    """
    word = word.lower()

    # Отримуємо список кандидатів
    candidates = get_candidates(word, vocab)

    # Якщо кандидатів немає, повертаємо оригінальне слово
    if not candidates:
        return word

    # Обираємо кандидата з найвищою ймовірністю
    best_candidate = max(candidates, key=lambda c: get_probability(c, freqs, total_words))

    return best_candidate

# Приклад використання
corrected_word = autocorrect('hamletz', vocab, freqs, total_words)
print(f"Виправлення для 'hamletz': {corrected_word}")

corrected_word_2 = autocorrect('caeser', vocab, freqs, total_words)
print(f"Виправлення для 'caeser': {corrected_word_2}")

Виправлення для 'hamletz': hamlet
Виправлення для 'caeser': caesar


In [16]:
def evaluate_autocorrect(test_data, vocab, freqs, total_words):
    """Оцінює точність системи автокорекції."""
    correct_predictions = 0
    total_tests = len(test_data)

    for misspelled, correct in test_data:
        prediction = autocorrect(misspelled, vocab, freqs, total_words)
        if prediction == correct:
            correct_predictions += 1
        print(f"Слово: '{misspelled}', Виправлення: '{prediction}', Правильно: '{correct}' -> {'OK' if prediction == correct else 'FAIL'}")

    accuracy = (correct_predictions / total_tests) * 100
    return accuracy

# Тестовий набір (помилка, правильне слово)
test_data = [
    ('thee', 'the'),
    ('quene', 'queen'),
    ('thou', 'thou'), # Правильне слово має бути повернуто як є
    ('learne', 'learn'),
    ('hamletz', 'hamlet'),
    ('kingdmoe', 'kingdom')
]

accuracy = evaluate_autocorrect(test_data, vocab, freqs, total_words)
print(f"\nПідсумкова точність системи: {accuracy:.2f}%")

Слово: 'thee', Виправлення: 'thee', Правильно: 'the' -> FAIL
Слово: 'quene', Виправлення: 'queen', Правильно: 'queen' -> OK
Слово: 'thou', Виправлення: 'thou', Правильно: 'thou' -> OK
Слово: 'learne', Виправлення: 'learn', Правильно: 'learn' -> OK
Слово: 'hamletz', Виправлення: 'hamlet', Правильно: 'hamlet' -> OK
Слово: 'kingdmoe', Виправлення: 'kingdom', Правильно: 'kingdom' -> OK

Підсумкова точність системи: 83.33%
